In [24]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
import warnings
warnings.filterwarnings("ignore")
import yaml
import torch
import segmentation_models_pytorch as smp
from torchvision import transforms
from PIL import Image

import boto3
import requests
import numpy as np
from urllib.request import urlopen

In [33]:

s3_resource = boto3.resource('s3')
bucketname = 'deepglobedata'

def download_files(bucket=bucketname, files=[], destination='.'):
    downloaded_files = []
    s3_resource = boto3.resource('s3')

    for file_key in files:
        location = os.path.join(destination, os.path.basename(file_key))
        if not os.path.exists(location):
            s3_resource.Object(bucket, file_key).download_file(location)
            downloaded_files.append(location)
    
    return downloaded_files

# Specify the keys for the model and configuration file
model_key = 'final_model.pth'
config_key = 'config.yaml'

# Download the model and configuration file to the current working directory
downloaded_files = download_files(bucket=bucketname, files=[model_key, config_key], destination=os.getcwd())

model_path = downloaded_files[0]  
config_path = downloaded_files[1] 


['/Users/noeliaotero/Documents/WeCloudData/Capstone_project/Deepglobe/notebooks/final_model.pth', '/Users/noeliaotero/Documents/WeCloudData/Capstone_project/Deepglobe/notebooks/config.yaml']


In [34]:
config = load_config(config_path)
model= load_model(config, model_path)

load state
movel eval


In [3]:
# Define neccesary functions
def load_config(config_path):
    # Load configuration from file
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

def build_model(config):
    # Get the selected model class from the mapping in YAML
    model_classes = config['MODEL_CLASSES']
    selected_model_class = getattr(smp, model_classes[config['MODEL']])

    # Create segmentation model with pretrained encoder
    model = selected_model_class(
        encoder_name=config['ENCODER'],
        encoder_weights=config['ENCODER_WEIGHTS'],
        classes=len(config['CLASSES']),
        activation=config['ACTIVATION'],
    )

    return model

def load_model(config, model_path):
    # Load the model
    model = build_model(config)
    try:
        state_dict = torch.load(model_path, map_location=torch.device('cpu'))
        print("load state")
        model.load_state_dict(state_dict)
    except Exception as e:
        print(f"An error occurred during model loading: {e}")
        # Add additional information or logging here
        raise e
    print("movel eval")
    model.eval()
    return model

In [12]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)
model_path = '../deploy_fastapi/final_model.pth'
config_path ='../deploy_fastapi/fastapi/config.yaml'

cpu


In [13]:
config = load_config(config_path) #load the config final with info about the model architecture
print("loading the model")
model = build_model(config)
state_dict = torch.load(model_path, map_location=torch.device(DEVICE))
model.load_state_dict(state_dict)
model.eval()

loading the model


DeepLabV3Plus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequentia

In [18]:
# Load one image sample
sample = '../data_images//test/100877_sat.jpg'

In [19]:
# Prepare the image for inference
img = Image.open(sample)
img = transforms.ToTensor()(img)
print("shape", img.shape)
# Add batch dimension
img = img.unsqueeze(0)
img = img.to(DEVICE)

shape torch.Size([3, 2448, 2448])


In [20]:
# Inference
pred_mask = model(img).detach().cpu().numpy() #takes a few seconds
pred_mask = pred_mask.squeeze()

In [21]:
print("output shape",pred_mask.shape)

output shape (7, 2448, 2448)
